In [2]:
import pyodbc
import pandas as pd
# import unittest
from helper.helper import Helper
from sparky_bc import Sparky
import os
import getpass

usr=getpass.getuser()
pass_usr=os.environ['PWD']

sp = Sparky(usr,"IMPALA_PROD", False, pass_usr)
hp = sp.helper

ResultZone = 'resultados_riesgos'


2022-02-04 09:21:00 - [WARNING] - No se encontro la carpeta "c:\Users\diecebal\Documents\Saldos_Diarios_Orquestado\test\logs" para guardar los logs


 ____  _____ __  __  ___ _____ _____ 
|  _ \| ____|  \/  |/ _ \_   _| ____|
| |_) |  _| | |\/| | | | || | |  _|  
|  _ <| |___| |  | | |_| || | | |___ 
|_| \_\_____|_|  |_|\___/ |_| |_____|
                                     
 ____  ____   _    ____  _  __
/ ___||  _ \ / \  |  _ \| |/ /
\___ \| |_) / _ \ | |_) | ' / 
 ___) |  __/ ___ \|  _ <| . \ 
|____/|_| /_/   \_\_| \_\_|\_\
                              



In [3]:
asigna_df_cn = hp.obtener_dataframe('select * from proceso.fechas_cierre')
fecha_cierre = asigna_df_cn.iloc[0]['fecha_corte']
print(fecha_cierre)

------------------------------------------------------------
  i    tipo    nombre    estado     hora_inicio   duracion   
------------------------------------------------------------
 1/1 DATAFRAME         ejecutando   09:21:23 AM             

2022-02-04 09:21:23 - [INFO] - 0 filas, 3 columnas, 00:00.3 consultando, 00:00.0 descargando, 00:00.0 convirtiendo


 1/1 DATAFRAME         finalizado   09:21:23 AM     00:00.4 
------------------------------------------------------------


IndexError: single positional indexer is out-of-bounds

In [1]:
import pandas as pd
import numpy as np
import sys
import pyodbc
import logging
from datetime import datetime

#Funcion para consultas en la base de datos
def runQuery(query):
    CONN_STR='DSN=IMPALA_PROD'
    cn = pyodbc.connect(CONN_STR, autocommit = True )
    cursor = cn.cursor()
    cursor.execute(query)

#Funcion para consultas en la base de datos guardando la informacion en un dataframe
def runQueryPandas(query):
    CONN_STR='DSN=IMPALA_PROD'
    cn = pyodbc.connect(CONN_STR, autocommit = True )
    data = pd.read_sql_query(query,cn)
    return data

In [2]:
hoy="22"
ayer="21"
corte="20211222"
querys_calculo=[
""" SELECT        CAST(sum(sld_cap_final)/1000000 AS INT) AS sk_hoy, CAST(sum(cv)/1000000 AS INT) AS cv_hoy FROM proceso_riesgos.insumo_saldos WHERE ingestion_day="""+hoy+""""""
,""" SELECT        CAST(sum(sld_cap_final)/1000000 AS INT) AS sk_ayer, CAST(sum(cv)/1000000 AS INT) AS cv_ayer FROM proceso_riesgos.insumo_saldos WHERE ingestion_day="""+ayer+""""""
,""" SELECT CAST(sum(sld_cap_act)/1000000 AS INT) as sk_hoy,
       CAST(sum(sld_cap_ant)/1000000 AS INT) as sk_ayer,
       CAST(sum(cv_act)/1000000 AS INT) as cv_hoy,
       CAST(sum(cv_ant)/1000000 AS INT) as cv_ayer FROM proceso_riesgos.variaciones_saldos_diarios """
,""" SELECT CAST(sum(sld_cap_act)/1000000 AS INT) as sk_hoy,
       CAST(sum(sld_cap_ant)/1000000 AS INT) as sk_ayer,
       CAST(sum(cv_act)/1000000 AS INT) as cv_hoy,
       CAST(sum(cv_ant)/1000000 AS INT) as cv_ayer FROM proceso_riesgos.saldos_diarios_erm_pre1 """
,""" SELECT CAST(sum(sld_cap_act)/1000000 AS INT) as sk_hoy,
       CAST(sum(sld_cap_ant)/1000000 AS INT) as sk_ayer,
       CAST(sum(cv_act)/1000000 AS INT) as cv_hoy,
       CAST(sum(cv_ant)/1000000 AS INT) as cv_ayer FROM proceso_riesgos.saldos_diarios_erm_pre2 """
,""" SELECT CAST(sum(sld_cap_act)/1000000 AS INT) as sk_hoy,
       CAST(sum(sld_cap_ant)/1000000 AS INT) as sk_ayer,
       CAST(sum(cv_act)/1000000 AS INT) as cv_hoy,
       CAST(sum(cv_ant)/1000000 AS INT) as cv_ayer FROM proceso_riesgos.saldos_diarios_new_"""+corte+""""""]

In [3]:
# nombre_bases=[
# """proceso_riesgos.insumo_saldos """
# ,"""proceso_riesgos.insumo_saldos """

# ,"""proceso_riesgos.variaciones_saldos_diarios """
# ,"""proceso_riesgos.saldos_diarios_erm_pre1 """
# ,"""proceso_riesgos.saldos_diarios_erm_pre2 """
# ,"""proceso_riesgos.saldos_diarios_new """]

In [4]:
def consultar(dataframe):
    dataframe_salida = pd.DataFrame()
    for x in querys_calculo:
        temp_data=runQueryPandas(x)
        temp_position=querys_calculo.index(x)
        temp_data.insert(0,"TABLA",nombre_bases[temp_position])
        print(temp_data)
        dataframe_salida=pd.concat([dataframe_salida,temp_data])
        print(dataframe_salida.head())
    return dataframe_salida

In [5]:
# salida = pd.DataFrame()
# salida_total= consultar(salida)
# salida_total.to_excel("saldos_tabla.xlsx")

In [6]:
querys_calculo=[
"SELECT COUNT(*) from PROCESO_RIESGOS.preaprobados_productos_otros_agrupado",
"SELECT COUNT(*) from PROCESO_RIESGOS.preaprob_multivariado_full",
"SELECT COUNT(*) from PROCESO_RIESGOS.preaprob_productos_otros_full",
"SELECT COUNT(*) from PROCESO_RIESGOS.preaprobados_multivariado_agrupado",
"SELECT ingestion_year, ingestion_month, ingestion_day from PROCESO_RIESGOS.preaprobados_productos_otros_agrupado LIMIT 1",
"SELECT ingestion_year, ingestion_month, ingestion_day from PROCESO_RIESGOS.preaprob_multivariado_full LIMIT 1",
"SELECT ingestion_year, ingestion_month, ingestion_day from PROCESO_RIESGOS.preaprob_productos_otros_full LIMIT 1",
"SELECT ingestion_year, ingestion_month, ingestion_day from PROCESO_RIESGOS.preaprobados_multivariado_agrupado LIMIT 1"]

In [7]:
nombre_bases=[
"""preaprobados_productos_otros_agrupado """
,"""ppreaprob_multivariado_full """
,"""PROCESO_RIESGOS.preaprob_productos_otros_full """
,"""preaprobados_multivariado_agrupado """

,"""fechas 1 """
,"""fechas 2 """
,"""fechas 3 """
,"""fechas 4 """]

In [8]:
salida = pd.DataFrame()
salida_total= consultar(salida)
salida_total.to_excel("saldos_tabla.xlsx")

                                    TABLA  expr_1
0  preaprobados_productos_otros_agrupado     6334
                                    TABLA  expr_1
0  preaprobados_productos_otros_agrupado     6334
                          TABLA    expr_1
0  ppreaprob_multivariado_full   17561055
                                    TABLA    expr_1
0  preaprobados_productos_otros_agrupado       6334
0            ppreaprob_multivariado_full   17561055
                                            TABLA    expr_1
0  PROCESO_RIESGOS.preaprob_productos_otros_full   15641450
                                            TABLA    expr_1
0          preaprobados_productos_otros_agrupado       6334
0                    ppreaprob_multivariado_full   17561055
0  PROCESO_RIESGOS.preaprob_productos_otros_full   15641450
                                 TABLA    expr_1
0  preaprobados_multivariado_agrupado   10980546
                                            TABLA    expr_1
0          preaprobados_productos_otros_ag